In [ ]:
# First Time data pull
'''

# Import data
import pandas as pd
import os
from googletrans import Translator
#os.chdir('d:\\')
comment_list = []
with open('LEI_COMMENT.txt') as f:
    comment_list=[line.rstrip('\n') for line in f]
with open('LEI_RATING.txt') as f:
    rating_list = [line.rstrip('\n') for line in f]
lei_df = pd.DataFrame()
lei_df['comment'] = pd.Series(comment_list)
lei_df['lei_rating'] = pd.Series(rating_list)
translator = Translator()

# Translate French to English

def f(x):
    try:
        return translator.translate(x, dest='en').text
    except:
        return 'translation not available'

lei_df['translated_comment'] = lei_df['comment'].apply(f)

# Save file for future use
lei_df.to_csv('LEI_DF.txt')

EXPORT TO C:\Users\khatwd2\Documents\lei_sow.txt OF DEL MODIFIED BY NOCHARDEL 
select 
a.model_score_rank_no
,b.rownumber
 from 
(select * from EDW.MAP_MODEL_SCORE where model_id = 1214 and efectv_dt = '2017-08-31')a
inner join
(select comment,cust_id,ROW_NUMBER() OVER () rownumber from  CHERA23.ENTPR_HACK_LEI) b
on a.cust_id = b.cust_id

'''

In [ ]:
# random
import random

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

# Clean text to remove stop words and lower case everything
import re
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()


def clean_data(input_str):
    try:
        input_str = re.sub('<[^<]+?>', ' ', input_str)
        out = re.sub('[^A-Za-z0-9]+', ' ', input_str.lower())
        out = ' '.join([i for i in out.split() if i not in stop_words])
        #out = ' '.join([ wordnet_lemmatizer.lemmatize(porter_stemmer.stem(i)) for i in out.split() if i not in stop_words])
        return out
    except:
        return input_str

In [ ]:
#Load previously formed dataframe
import pandas as pd
import numpy as np
import csv
lei_df = pd.read_csv('LEI_DF.txt', encoding ='latin-1',index_col=0)
def f1(x):
    if x['translated_comment'] =='translation not available':
           return x['comment']
    else:
        return x['translated_comment']

lei_df['final_comment']=lei_df.apply(f1, axis=1)

# Copy dataframe for model building
lei_df_model= lei_df.copy()

# Clean data
lei_df_model['final_comment_clean']=lei_df_model['final_comment'].apply(lambda x: clean_data(x))

In [ ]:
# Load judith's labeled data
lei_label_df = pd.read_excel('Comments_Set2.xlsx', index_col = 'Index')

# seperate out labeled and unlabeled samples
labeled_samples = list(lei_label_df.index.values)
unlabeled_samples = [elem for elem in list(lei_df_model.index.values) if elem not in labeled_samples]

# generate test set based on unlabeled samples
lei_df_model_test = lei_df_model.iloc[unlabeled_samples].copy()

#change encoding to UTF-8
def encode_f(x):
    try:
        return x.encode('utf-8')
    except:
        return 'not available'.encode('utf-8')
    
lei_df_model['final_comment_clean']=lei_df_model['final_comment_clean'].apply(encode_f)

In [ ]:
# get the pretrained glove word embeddings and helper functions
filename = 'glove.6B\\glove.6B.50d.txt'
words = pd.read_table(filename, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

def vec(w):
  return words.loc[w].as_matrix()

def get_sent_embeddings(x):
    sent_vec = []
    for word in x.split():
        try:
            sent_vec.append(vec(str(word)[2:-1]))
        except:
            sent_vec.append(vec('unk'))
    sent_vec = np.array(sent_vec)
    sent_vec =  np.mean(sent_vec, axis=0)
    return sent_vec

In [ ]:
# convert the comments' word to vector
l=lei_df_model['final_comment_clean'].apply(get_sent_embeddings)

In [ ]:
# create a dataframe out of the word embeddings
numberOfRows = lei_df_model.shape[0]
column_names = []
for i in range(50):
    column_names.append('vec'+str(i))
# create dataframe
d = pd.DataFrame(index=np.arange(0, numberOfRows), columns = column_names)

# now fill it up row by row
for x in np.arange(0, numberOfRows):
    #loc or iloc both work here since the index is natural numbers
    d.loc[x] = l[x]

In [ ]:
df_labeled = d.loc[labeled_samples]
df_labeled['label_sent'] = lei_label_df['Sentiment']
df_labeled['label_td'] = lei_label_df['TD only']
df_labeled.dropna(inplace = True)
z = {1:1,0:0,-1:2}
df_labeled['label_sent']=df_labeled['label_sent'].map(z)

In [ ]:
df_labeled['label_td'].value_counts()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn import svm

from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.svm import SVC

X = df_labeled.drop(['label_sent','label_td'], axis = 1).as_matrix() 
#y =  df_labeled['label_sent'].as_matrix() 
y =  df_labeled['label_td'].as_matrix() 

# semisupervised learning try 1

label_prop_model = LabelPropagation()
rng = np.random.RandomState(0)
random_unlabeled_points = rng.rand(len(y)) < 0.5
labels = np.copy(y)
labels[random_unlabeled_points] = -1
label_prop_model.fit(X, labels)
predictions = label_prop_model.predict(X)
plot = skplt.metrics.plot_confusion_matrix(y, predictions, normalize=False)
print(pd.Series(predictions).value_counts())

# Semi supervised learning try 2
label_prop_model = LabelPropagation()
rng = np.random.RandomState(0)
random_unlabeled_points = rng.rand(len(y)) < 0.3
labels = np.copy(y)
labels[random_unlabeled_points] = -1
label_prop_model.fit(X, labels)
predictions = label_prop_model.predict(X)
plot = skplt.metrics.plot_confusion_matrix(y, predictions, normalize=False)
print(pd.Series(predictions).value_counts())

# normal supervised algo using 30% unlabeled points
basemodel = OneVsRestClassifier(SVC(kernel='linear',probability=True))
predictions = basemodel.fit(X,y).predict(X)
plot = skplt.metrics.plot_confusion_matrix(y, predictions, normalize=False)
print(pd.Series(predictions).value_counts())

plt.show()

In [ ]:
sow_rank=pd.read_csv('lei_sow.txt',  encoding = 'latin-1',error_bad_lines=False, names = ['model_rank','model_sc'],index_col = 2)

In [ ]:
sow_rank.head()

In [ ]:
result = sow_rank.join(lei_label_df, how='inner')
predictions = pd.DataFrame(label_prop_model.predict(X), index = df_labeled.index,columns= ['prediction'])
result =result.join(predictions,how = 'inner')

In [ ]:
result

In [ ]:
result['sow_model_prediction']= result['model_sc'].apply(lambda x: (x>=0.6)*1)

In [ ]:
plot = skplt.metrics.plot_confusion_matrix(result[result['TD only']==1]['sow_model_prediction'], result[result['TD only']==1]['prediction'], normalize=False)
plt.show()

In [ ]:
#plt.scatter(result[result['TD only']==1]['sow_model_prediction'],result[result['TD only']==1]['prediction'])
#skplt.metrics.plot_confusion_matrix(result[result['TD only']==1]['sow_model_prediction'],result[result['TD only']==1]['prediction'], normalize=False)

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('labeled dataset.pdf') as pdf:
    # Get current size
    fig_size = plt.rcParams["figure.figsize"]
    # Set figure width to 12 and height to 9
    fig_size[0] = 12
    fig_size[1] = 12
    plt.rcParams["figure.figsize"] = fig_size
    
    plt.hist([result[result['TD only']==1]['sow_model_prediction'],result[result['TD only']==1]['prediction']],label =['sow model','our model'])
    plt.xlabel("Predictions")
    plt.ylabel("TD Only Customers") 
    plt.legend()
    pdf.savefig()
    plt.close()

In [ ]:
predictions = pd.Series(label_prop_model.predict(d.dropna().as_matrix()), index = d.dropna().index)

In [ ]:
predictions.value_counts()

In [ ]:
result2 =d.dropna().join(sow_rank,how = 'inner')

In [ ]:
result2['predictions']= predictions

In [ ]:
result2['sow_model_prediction']= result2['model_sc'].apply(lambda x: (x>=0.6)*1)

In [ ]:
plot = skplt.metrics.plot_confusion_matrix(result2['sow_model_prediction'], result2['predictions'], normalize=False)
plt.show()

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('Full data set.pdf') as pdf:
    # Get current size
    fig_size = plt.rcParams["figure.figsize"]
    # Set figure width to 12 and height to 9
    fig_size[0] = 12
    fig_size[1] = 12
    plt.rcParams["figure.figsize"] = fig_size
    
    plt.hist([result2['sow_model_prediction'],result2['predictions']],label =['sow model','our model'])
    plt.xlabel("Predictions")
    plt.ylabel("Customers")
    plt.legend()
    pdf.savefig()
    plt.close()

In [ ]:


'''
y_30 = np.copy(y)
y_30[rng.rand(len(y)) < 0.3] = -1
y_50 = np.copy(y)
y_50[rng.rand(len(y)) < 0.5] = -1
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors

ls30 = (label_propagation.LabelSpreading().fit(X, y_30),y_30)
ls50 = (label_propagation.LabelSpreading().fit(X, y_50),y_50)
ls100 = (label_propagation.LabelSpreading().fit(X, y), y)
rbf_svc = (svm.SVC(kernel='rbf').fit(X, y), y)

predictions = ls30.predict(X)
plot = skplt.metrics.plot_confusion_matrix(ytrue, predictions, normalize=True)

predictions = ls50.predict(X)
plot = skplt.metrics.plot_confusion_matrix(ytrue, predictions, normalize=True)

predictions = ls100.predict(X)
plot = skplt.metrics.plot_confusion_matrix(ytrue, predictions, normalize=True)

from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.preprocessing import StandardScaler
datasets = [make_moons(noise=0.3, random_state=0)]
for ds_cnt, ds in enumerate(datasets):
    # preprocess dataset, split into training and test part
    X, ytrue = ds
    X = StandardScaler().fit_transform(X)
    

# label a few points 
labeled_N = 240
ys = np.array([-1]*len(ytrue)) # -1 denotes unlabeled point
random_labeled_points = np.union1d(np.random.choice(np.ravel(np.where(ytrue == -1)), 50), 
                                   np.random.choice(np.ravel(np.where(ytrue == 1)), 50))
random_labeled_points = np.union1d(random_labeled_points,np.random.choice(np.ravel(np.where(ytrue == 0)), 50)) 

ys[random_labeled_points] = ytrue[random_labeled_points]

# Basemodel using Scikit learn
basemodel =  OneVsRestClassifier(SVC(kernel='linear',probability=True))
#basemodel = SGDClassifier(loss='log', penalty='l1') 
predictions = basemodel.fit(X[random_labeled_points, :], ys[random_labeled_points]).predict(X)
plot = skplt.metrics.plot_confusion_matrix(ytrue, predictions, normalize=True)


'''


In [ ]:
lei_df_model['final_comment_clean'][1200]